In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
import pandas as pd
import cv2
import os
import yaml
from tqdm import tqdm

In [2]:
!git clone https://github.com/WongKinYiu/yolov7
!pip install -r yolov7/requirements.txt
!pip uninstall -y wandb
!pip install yt-dlp
!pip install ffmpeg

Cloning into 'yolov7'...
Updating files:  72% (75/104)
Updating files:  73% (76/104)
Updating files:  74% (77/104)
Updating files:  75% (78/104)
Updating files:  76% (80/104)
Updating files:  77% (81/104)
Updating files:  78% (82/104)
Updating files:  79% (83/104)
Updating files:  80% (84/104)
Updating files:  81% (85/104)
Updating files:  82% (86/104)
Updating files:  83% (87/104)
Updating files:  84% (88/104)
Updating files:  85% (89/104)
Updating files:  86% (90/104)
Updating files:  87% (91/104)
Updating files:  88% (92/104)
Updating files:  89% (93/104)
Updating files:  90% (94/104)
Updating files:  91% (95/104)
Updating files:  92% (96/104)
Updating files:  93% (97/104)
Updating files:  94% (98/104)
Updating files:  95% (99/104)
Updating files:  96% (100/104)
Updating files:  97% (101/104)
Updating files:  98% (102/104)
Updating files:  99% (103/104)
Updating files: 100% (104/104)
Updating files: 100% (104/104), done.
  Consider adding this directory to PATH or, if you prefer to 

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



Defaulting to user installation because normal site-packages is not writeable
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=a610b768f92bbe2b1f65cefc98fa77702c27c2dd3eb78497d3e363c71b846889
  Stored in directory: c:\users\ahmed\appdata\local\pip\cache\wheels\1d\57\24\4eff6a03a9ea0e647568e8a5a0546cdf957e3cf005372c0245
Successfully built ffmpeg


In [3]:
classes = np.array([
    'A10','A400M','AG600','AV8B',
    'B1','B2','B52','Be200',
    'C130','C17','C5','E2','EF2000',
    'F117','F14','F15','F16','F18','F22','F35','F4',
    'JAS39','MQ9','Mig31','Mirage2000','RQ4','Rafale',
    'SR71','Su34','Su57',
    'Tu160','Tu95','Tornado',
    'U2','US2', 'V22','XB70','YF23','Vulcan','J20']
)

In [4]:
with open('./yolov7/detect.py') as f:
    data_lines = f.read()
data_lines = data_lines.replace('line_thickness=1', 'line_thickness=4')
with open('./yolov7/detect.py', mode='w') as f:
    f.write(data_lines)

In [6]:
csv_paths = glob.glob('D:\\AirDefense\\dataset/*.csv')
jpg_paths = glob.glob('D:\\AirDefense\\dataset/*.jpg')
csv_paths.sort()
jpg_paths.sort()
print('number of images:',len(csv_paths))


os.makedirs('./yolov7/data/train/images', exist_ok=True)
os.makedirs('./yolov7/data/train/labels', exist_ok=True)

os.makedirs('./yolov7/data/valid/images', exist_ok=True)
os.makedirs('./yolov7/data/valid/labels', exist_ok=True)

os.makedirs('./yolov7/data/test/images', exist_ok=True)
os.makedirs('./yolov7/data/test/labels', exist_ok=True)

number of images: 8341


In [7]:
for i, (csv_path, jpg_path) in enumerate(tqdm(zip(csv_paths, jpg_paths))):
    
    annotations = np.array(pd.read_csv(csv_path))
    
    if os.path.basename(csv_path)[0] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b']:
        jpg_file_path = './yolov7/data/train/images/' + os.path.basename(jpg_path)
        txt_file_path = './yolov7/data/train/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
        shutil.copy(jpg_path, jpg_file_path)
    elif os.path.basename(csv_path)[0] in ['c', 'd', 'e']:
        jpg_file_path = './yolov7/data/valid/images/' + os.path.basename(jpg_path)
        txt_file_path = './yolov7/data/valid/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
        shutil.copy(jpg_path, jpg_file_path)
    else:
        jpg_file_path = './yolov7/data/test/images/' + os.path.basename(jpg_path)
        txt_file_path = './yolov7/data/test/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
        shutil.copy(jpg_path, jpg_file_path)
    
    with open(txt_file_path, mode='w') as f:
        try:
            for annotation in annotations:
                width = annotation[1]
                height = annotation[2]
                class_name = annotation[3]
                xmin = annotation[4]
                ymin = annotation[5]
                xmax = annotation[6]
                ymax = annotation[7]
                x_center = 0.5*(xmin+xmax)
                y_center = 0.5*(ymin+ymax)
                b_width = xmax - xmin
                b_height= ymax - ymin
                class_num = np.where(classes==class_name)[0][0]
                output_string = '{} {} {} {} {}\n'.format(class_num,
                                                        x_center/width,
                                                        y_center/height,
                                                        b_width/width,
                                                        b_height/height)
                f.write(output_string)
        except:
            print(txt_file_path)
            0/0

print('test:',len(glob.glob('./yolov7/data/test/labels/*.txt')))
print('valid:',len(glob.glob('./yolov7/data/valid/labels/*.txt')))
print('train',len(glob.glob('./yolov7/data/train/labels/*.txt')))

8341it [12:25, 11.18it/s]


test: 566
valid: 1543
train 6232


In [8]:
classes_string = ', '.join([f'"{c}"' for c in classes])
with open('./yolov7/data/mad.yaml', mode='w') as f:
    yaml_string='train: ./yolov7/data/train\n'\
              + 'val: ./yolov7/data/valid\n'\
              + 'test: ./yolov7/data/test\n'\
              + f'nc: {len(classes)}\n'\
              + f'names: [{classes_string}]'
    f.write(yaml_string)
print(yaml_string)

train: ./yolov7/data/train
val: ./yolov7/data/valid
test: ./yolov7/data/test
nc: 40
names: ["A10", "A400M", "AG600", "AV8B", "B1", "B2", "B52", "Be200", "C130", "C17", "C5", "E2", "EF2000", "F117", "F14", "F15", "F16", "F18", "F22", "F35", "F4", "JAS39", "MQ9", "Mig31", "Mirage2000", "RQ4", "Rafale", "SR71", "Su34", "Su57", "Tu160", "Tu95", "Tornado", "U2", "US2", "V22", "XB70", "YF23", "Vulcan", "J20"]


In [9]:
hyp = {
    'lr0': 0.0002,  # initial learning rate (SGD=1E-2, Adam=1E-3)
    'lrf': 0.001,  # final OneCycleLR learning rate (lr0 * lrf)
    'momentum': 0.937,  # SGD momentum/Adam beta1
    'weight_decay': 0.0005,  # optimizer weight decay 5e-4
    'warmup_epochs': 2.0,  # warmup epochs (fractions ok)
    'warmup_momentum': 0.8,  # warmup initial momentum
    'warmup_bias_lr': 0.0001,  # warmup initial bias lr
    'box': 0.05,  # box loss gain
    'cls': 0.3,  # cls loss gain
    'cls_pw': 1.0,  # cls BCELoss positive_weight
    'obj': 0.7,  # obj loss gain (scale with pixels)
    'obj_pw': 1.0,  # obj BCELoss positive_weight
    'iou_t': 0.20,  # IoU training threshold
    'anchor_t': 4.0,  # anchor-multiple threshold
    # anchors: 3  # anchors per output layer (0 to ignore)
    'fl_gamma': 0.0,  # focal loss gamma (efficientDet default gamma=1.5)
    'hsv_h': 0.015,  # image HSV-Hue augmentation (fraction)
    'hsv_s': 0.7,  # image HSV-Saturation augmentation (fraction)
    'hsv_v': 0.4,  # image HSV-Value augmentation (fraction)
    'degrees': 0.5,  # image rotation (+/- deg)
    'translate': 0.25,  # image translation (+/- fraction)
    'scale': 0.45,  # image scale (+/- gain)
    'shear': 0.5,  # image shear (+/- deg)
    'perspective': 0.0,  # image perspective (+/- fraction), range 0-0.001
    'flipud': 0.15,  # image flip up-down (probability)
    'fliplr': 0.5,  # image flip left-right (probability)
    'mosaic': 0.8,  # image mosaic (probability)
    'mixup': 0.35,  # image mixup (probability)
    'copy_paste': 0,  # image copy paste (probability)
    'paste_in': 0,  # image copy paste (probability), use 0 for faster training
    'loss_ota': 1, # use ComputeLossOTA, use 0 for faster training
}
with open('./yolov7/data/hyp.yaml', 'w') as f:
    yaml.dump(hyp, f)

In [18]:
# YOLOv-W6
# os.makedirs('./yolov7/checkpoints', exist_ok=True)
# shutil.copy('/yolov7-military-plane/best.pt', './yolov7/checkpoints/yolov7-w6.pt')
# download weights from url https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6.pt

Cloning into 'yolov7-w6.pt'...
remote: Not Found
fatal: repository 'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6.pt/' not found


In [19]:
"""
# colab A100
!python ./yolov7/train_aux.py \
  --batch-size 20 \
  --img-size 1280 \
  --epochs 55 \
  --name yolov7_mad_12 \
  --adam \
  --data ./yolov7/data/mad.yaml \
  --cfg ./yolov7/cfg/training/yolov7-w6.yaml \
  --weights ./yolov7/checkpoints/yolov7-w6.pt \
  --v5-metric \
  --hyp ./yolov7/data/hyp.yaml
"""

'\n# colab A100\n!python ./yolov7/train_aux.py   --batch-size 20   --img-size 1280   --epochs 55   --name yolov7_mad_12   --adam   --data ./yolov7/data/mad.yaml   --cfg ./yolov7/cfg/training/yolov7-w6.yaml   --weights ./yolov7/checkpoints/yolov7-w6.pt   --v5-metric   --hyp ./yolov7/data/hyp.yaml\n'

In [20]:
!python ./yolov7/test.py \
  --batch-size 4 \
  --img-size 1400 \
  --name yolov7_mad_12 \
  --data ./yolov7/data/mad.yaml \
  --weights ./yolov7/checkpoints/yolov7-w6.pt \
  --v5-metric

Namespace(weights=['./yolov7/checkpoints/yolov7-w6.pt'], data='./yolov7/data/mad.yaml', batch_size=4, img_size=1400, conf_thres=0.001, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7_mad_12', exist_ok=False, no_trace=False, v5_metric=True)

YOLOR  v0.1-116-g8c0bf3f torch 1.13.0+cpu CPU

Model Summary: 343 layers, 70394300 parameters, 70394300 gradients
C:\Users\ahmed\AppData\Roaming\Python\Python39\site-packages\torch\nn\modules\module.py:673: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\build\aten\src\ATen/core/TensorBody.h:485.)
  if param.grad is not None:

Scanning images:   0%|          | 0/1543 [00:00<?, ?it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 1 found, 0 missing, 0 empty, 0 corrupted:   0%|          | 1/1543 [00:00<19:52,


Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

Testing with YOLOv5 AP metric...


val: Scanning 'yolov7\data\valid\labels' images and labels... 312 found, 0 missing, 0 empty, 0 corrupted:  20%|##        | 312/1543 [00:23<01:31, 13.43it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 314 found, 0 missing, 0 empty, 0 corrupted:  20%|##        | 314/1543 [00:23<01:50, 11.17it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 316 found, 0 missing, 0 empty, 0 corrupted:  20%|##        | 316/1543 [00:23<01:40, 12.17it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 319 found, 0 missing, 0 empty, 0 corrupted:  21%|##        | 319/1543 [00:23<01:23, 14.74it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 322 found, 0 missing, 0 empty, 0 corrupted:  21%|##        | 322/1543 [00:24<01:15, 16.16it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 324 found, 0 missing, 0 empty, 0 corrupted:  21%|##        | 324/1543 [00:24<01:16, 16.01it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels

val: Scanning 'yolov7\data\valid\labels' images and labels... 940 found, 0 missing, 0 empty, 0 corrupted:  61%|######    | 940/1543 [00:56<00:25, 23.55it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 943 found, 0 missing, 0 empty, 0 corrupted:  61%|######1   | 943/1543 [00:56<00:29, 20.47it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 946 found, 0 missing, 0 empty, 0 corrupted:  61%|######1   | 946/1543 [00:57<00:27, 21.51it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 950 found, 0 missing, 0 empty, 0 corrupted:  62%|######1   | 950/1543 [00:57<00:23, 25.19it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 953 found, 0 missing, 0 empty, 0 corrupted:  62%|######1   | 953/1543 [00:57<00:24, 24.40it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels... 958 found, 0 missing, 0 empty, 0 corrupted:  62%|######2   | 958/1543 [00:57<00:19, 30.01it/s]
val: Scanning 'yolov7\data\valid\labels' images and labels

In [22]:
for path in sorted(glob.glob('runs/test/yolov7_mad_12/*.png')):
    image = cv2.imread(path)[:,:,::-1]
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    plt.show()